In [18]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from statsmodels.stats.contingency_tables import mcnemar
from dataset_tools import OwnDataset, make_train_test_val_split
from models import LatentClassifier_2, VAE_CELL_CNN, CELL_CNN_CLASSIFIER

cuda = torch.cuda.is_available()
print(cuda)

False


In [20]:
tf = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])

dataset = OwnDataset(transform=tf, path=r"/Users/rasmusjensen/Documents/02466-Cell-Profiling/data_subset")
train_subset, test_subset, val_subset = make_train_test_val_split(dataset)

train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=64, shuffle=True)

print(len(dataset), len(train_loader), len(test_loader), len(val_loader))


7774 78 25 20


In [21]:
MODEL_DIR = "trained_models/"

latent_dim_256 = LatentClassifier_2(hidden_dim=512, latent_dim=256, num_classes=13)
latent_dim_256.load_state_dict(torch.load(MODEL_DIR + f"{latent_dim_256.__class__.__name__}_latent{latent_dim_256.latent_dim}_mean_best_model.pth", map_location=torch.device('cpu')))

latent_dim_2 = LatentClassifier_2(hidden_dim=4, latent_dim=2, num_classes=13)
latent_dim_2.load_state_dict(torch.load(MODEL_DIR + f"{latent_dim_2.__class__.__name__}_latent{latent_dim_2.latent_dim}_mean_best_model.pth", map_location=torch.device('cpu')))

Cell_CNN_Classifier = CELL_CNN_CLASSIFIER(input_dim=(3, 68, 68), hidden_dim=128, num_classes=13)
Cell_CNN_Classifier.load_state_dict(torch.load(MODEL_DIR + f"{Cell_CNN_Classifier.__class__.__name__}_best_model.pth", map_location=torch.device('cpu')))

Vae_Cell_CNN_256 = VAE_CELL_CNN(input_dim=(3, 68, 68), hidden_dim=512, latent_dim=256)
Vae_Cell_CNN_256.load_state_dict(torch.load(MODEL_DIR + f"{Vae_Cell_CNN_256.__class__.__name__}_latent{Vae_Cell_CNN_256.latent_dim}_mean_best_model.pth", map_location=torch.device('cpu')))

Vae_Cell_CNN_2 = VAE_CELL_CNN(input_dim=(3, 68, 68), hidden_dim=4, latent_dim=2)
Vae_Cell_CNN_2.load_state_dict(torch.load(MODEL_DIR + f"{Vae_Cell_CNN_2.__class__.__name__}_latent{Vae_Cell_CNN_2.latent_dim}_mean_best_model.pth", map_location=torch.device('cpu')))

models = [latent_dim_256, latent_dim_2, Cell_CNN_Classifier]


In [22]:

batch_accuracy_256_latent = []
batch_accuracy_2_latent = []
batch_accuracy_CNN = []

for image, moa in train_loader:
    latent_output = Vae_Cell_CNN_256(image)
    x_hat, mu, sigma = latent_output["x_hat"], latent_output["mu"], latent_output["sigma"]
    pred_256 = latent_dim_256(mu)
    batch_accuracy_256_latent.append((pred_256.argmax(1) == moa).cpu().numpy())

    latent_output = Vae_Cell_CNN_2(image)
    x_hat, mu, sigma = latent_output["x_hat"], latent_output["mu"], latent_output["sigma"]
    pred_2 = latent_dim_2(mu)
    batch_accuracy_2_latent.append((pred_2.argmax(1) == moa).cpu().numpy())

    batch_accuracy_CNN.append((Cell_CNN_Classifier(image).argmax(1) == moa).cpu().numpy())

batch_accuracy_CNN = np.concatenate(batch_accuracy_CNN)
batch_accuracy_256_latent = np.concatenate(batch_accuracy_256_latent)
batch_accuracy_2_latent = np.concatenate(batch_accuracy_2_latent)


In [30]:
def compute_mcnemar(acc1, acc2):
    n00 = n01 = n10 = n11 = 0
    for i in range(len(acc1)):
        if acc1[i] and acc2[i]:
            n11 += 1
        elif acc1[i] and not acc2[i]:
            n10 += 1
        elif not acc1[i] and acc2[i]:
            n01 += 1
        else:
            n00 += 1
    confusion_matrix = [[n00, n01], [n10, n11]]
    print(f'Confusion Matrix: {confusion_matrix}')
    result = mcnemar(confusion_matrix, exact=True)
    print(f'statistic={result.statistic}, p-value={result.pvalue}')
    if result.pvalue < 0.05:
        print('Significant difference in performance (reject H0)')
    else:
        print('No significant difference in performance (fail to reject H0)')


In [31]:
# Perform McNemar's tests
print("CNN vs Latent 256")
print("----------------")
compute_mcnemar(batch_accuracy_CNN, batch_accuracy_256_latent)
print(" ")

print("Latent 256 vs Latent 2")
print("----------------")
compute_mcnemar(batch_accuracy_256_latent, batch_accuracy_2_latent)
print(" ")

print("Latent 2 vs CNN")
print("----------------")
compute_mcnemar(batch_accuracy_2_latent, batch_accuracy_CNN)
print(" ")


CNN vs Latent 256
----------------
Confusion Matrix: [[4968, 6], [1, 0]]
statistic=1.0, p-value=0.125
No significant difference in performance (fail to reject H0)
 
Latent 256 vs Latent 2
----------------
Confusion Matrix: [[4969, 0], [6, 0]]
statistic=0.0, p-value=0.03125
Significant difference in performance (reject H0)
 
Latent 2 vs CNN
----------------
Confusion Matrix: [[4974, 1], [0, 0]]
statistic=0.0, p-value=1.0
No significant difference in performance (fail to reject H0)
 


In [2]:
from statsmodels.stats.contingency_tables import mcnemar

def compute_mcnemar(acc1, acc2):
    n00 = n01 = n10 = n11 = 0
    for i in range(len(acc1)):
        if acc1[i] == 1 and acc2[i] == 1:
            n11 += 1
        elif acc1[i] == 1 and acc2[i] == 0:
            n10 += 1
        elif acc1[i] == 0 and acc2[i] == 1:
            n01 += 1
        else:
            n00 += 1
    confusion_matrix = [[n00, n01], [n10, n11]]
    print(f'Confusion Matrix: {confusion_matrix}')
    result = mcnemar(confusion_matrix, exact=True)
    print(f'statistic={result.statistic}, p-value={result.pvalue}')
    if result.pvalue < 0.05:
        print('Significant difference in performance (reject H0)')
    else:
        print('No significant difference in performance (fail to reject H0)')


#acc1 = [1, 0, 1, 1, 0, 1, 0, 0, 1, 1]  
#acc2 = [1, 0, 1, 0, 0, 1, 0, 0, 1, 1]  
#compute_mcnemar(acc1, acc2)

compute_mcnemar(batch_accuracy_CNN, batch_accuracy_256_latent)


Confusion Matrix: [[4, 0], [1, 5]]
statistic=0.0, p-value=1.0
No significant difference in performance (fail to reject H0)


In [3]:
compute_mcnemar(batch_accuracy_CNN, batch_accuracy_256_latent)

NameError: name 'batch_accuracy_CNN' is not defined